## IBM Applied Data Science - Capstone Project

## Opening new shopping mall in Mumbai, India

# 1. Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20201017134407-0000
KERNEL_ID = f7582931-775c-43a1-98e8-020a8d8ad140
Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py37he5f6b98_2         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                   

In [2]:
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
!conda install -c conda-forge folium=0.5.0 --yes
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    attrs-20.2.0               |     pyh9f0ad1d_0          41 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py37hb5d75c8_1001         346 KB  conda-forge
    cffi-1.14.3                |   py37he30daa8_0         223 KB
    chardet-3.0.4              |py37he5f6b98_1008         170 KB  conda-forge
    cryptography-3.1.1         |   py37hff6837a_1         616 KB  conda-forge
    entrypoints-0.3            |py37hc8dfbb8_1002          13 KB  conda-forge
    f

pycparser-2.20       | 94 KB     | ##################################### | 100% 
numpy-1.19.2         | 5.2 MB    | ##################################### | 100% 
libcblas-3.8.0       | 11 KB     | ##################################### | 100% 
altair-4.1.0         | 614 KB    | ##################################### | 100% 
importlib_metadata-2 | 3 KB      | ##################################### | 100% 
libgfortran-ng-7.5.0 | 22 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


# 2. Import Data

In [3]:
markup  = requests.get('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai').text

In [4]:
soup = BeautifulSoup(markup, 'html.parser')

In [5]:
locationList = []
neighborhoodList = []
latitudeList=[]
longitudeList=[]

In [6]:
xtract = soup.find('table')
tmp = xtract.find_all('tr')

for row in tmp:
    cells=row.find_all('td')
    if(len(cells) > 0):
        neighborhoodList.append(cells[0].text.rstrip('\n'))
        locationList.append(cells[1].text.rstrip('\n'))
        latitudeList.append(cells[2].text.rstrip('\n'))
        longitudeList.append(cells[3].text.rstrip('\n'))

In [7]:
# create a new DataFrame 
df1 = pd.DataFrame({"Location": locationList,
                   "Neighborhood": neighborhoodList,
                   "Latitude": latitudeList,
                   "Longitude": longitudeList})

df1['Latitude']=pd.to_numeric(df1['Latitude'])
df1['Longitude']=pd.to_numeric(df1['Longitude'])

df1.head()

,Location,Neighborhood,Latitude,Longitude
0,"Andheri,Western Suburbs",Amboli,19.129300,72.843400
1,Western Suburbs,"Chakala, Andheri",19.111388,72.860833
2,"Andheri,Western Suburbs",D.N. Nagar,19.124085,72.831373
3,"Andheri,Western Suburbs",Four Bungalows,19.124714,72.827210
4,"Andheri,Western Suburbs",Lokhandwala,19.130815,72.829270


In [8]:
df1.count()

Location        93
Neighborhood    93
Latitude        93
Longitude       93
dtype: int64

# 3. Retrieve coordinates of Mumbai, India

In [9]:
# get the coordinates of Mumbai, India
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


# 4. Create map

In [10]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, location, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Location'], df1['Neighborhood']):
    label = '{}, {}'.format(location, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

# 5. Retrieve neighborhood venues from Foursquare

In [37]:
CLIENT_ID = 'X1BMWZGMAMWCTC3GX1R5JKG0ZETKZCDWYPNEUFAO04KONPIU' # your Foursquare ID
CLIENT_SECRET = 'U4HXXYQML3MS4M0EFB2DS3MNU4ZI3MYTDYKFOPO5KY3BE0JD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1BMWZGMAMWCTC3GX1R5JKG0ZETKZCDWYPNEUFAO04KONPIU
CLIENT_SECRET:U4HXXYQML3MS4M0EFB2DS3MNU4ZI3MYTDYKFOPO5KY3BE0JD


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
mumbai_venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )
print(mumbai_venues.shape)
mumbai_venues.head()

Amboli
Chakala, Andheri
D.N. Nagar
Four Bungalows
Lokhandwala
Marol
Sahar
Seven Bungalows
Versova
Mira Road
Bhayandar
Uttan
Bandstand Promenade
Kherwadi
Pali Hill
I.C. Colony
Gorai
Dahisa
Aarey Milk Colony
Bangur Nagar
Jogeshwari West
Juhu
Charkop
Poisar
Mahavir Nagar
Thakur village
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Kalina
Naigaon
Nalasopara
Virar
Irla
Vile Parle
Bhandup
Amrut Nagar
Asalfa
Pant Nagar
Kanjurmarg
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Indian Institute of Technology Bombay campus
Vidyavihar
Vikhroli
Chembur
Deonar
Mankhurd
Mahul
Agripada
Altamount Road
Bhuleshwar
Breach Candy
Carmichael Road
Cavel
Churchgate
Cotton Green
Cuffe Parade
Cumbala Hill
Currey Road
Dhobitalao
Dongri
Kala Ghoda
Kemps Corner
Lower Parel
Mahalaxmi
Mahim
Malabar Hill
Marine Drive
Marine Lines
Mumbai Central
Nariman Point
Prabhadevi
Sion
Walkeshwar
Worli
C.G.S. colony
Dagdi Chawl
Navy Nagar
Hindu colony
Ballard Estate
Chira Bazaar
Fanas Wadi
Chor Bazaar
Matunga
Parel
Gowalia Tank
D

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.1293,72.8434,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
1,Amboli,19.1293,72.8434,Merwans Cake shop,19.119300,72.845418,Bakery
2,Amboli,19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
3,Amboli,19.1293,72.8434,Hard Rock Cafe Andheri,19.135995,72.835335,American Restaurant
4,Amboli,19.1293,72.8434,Jaffer Bhai's Delhi Darbar,19.137714,72.845909,Mughlai Restaurant


In [40]:
#Group venues by Neighborhood
mumbai_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aarey Milk Colony,19,19,19,19,19,19
Agripada,100,100,100,100,100,100
Altamount Road,97,97,97,97,97,97
Amboli,86,86,86,86,86,86
Amrut Nagar,100,100,100,100,100,100
Asalfa,100,100,100,100,100,100
Ballard Estate,94,94,94,94,94,94
Bandstand Promenade,77,77,77,77,77,77
Bangur Nagar,100,100,100,100,100,100


In [41]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 223 uniques categories.


In [42]:
mumbai_venues.shape

(7014, 7)

In [43]:
mumbai_venues['Venue Category'].unique()[:50]

array(['Falafel Restaurant', 'Bakery', 'Indian Restaurant',
       'American Restaurant', 'Mughlai Restaurant', 'Pizza Place',
       'Brewery', 'Chinese Restaurant', 'Pub', 'Multiplex', 'Diner',
       'Ice Cream Shop', 'Café', 'BBQ Joint', 'Mediterranean Restaurant',
       'Sandwich Place', 'Lounge', 'Juice Bar', 'Coffee Shop', 'Spa',
       'Jewelry Store', 'Fast Food Restaurant', 'Comfort Food Restaurant',
       'Bar', 'Gym', 'Dessert Shop', 'Gym / Fitness Center',
       'Movie Theater', 'Residential Building (Apartment / Condo)',
       'Italian Restaurant', 'Deli / Bodega', "Women's Store",
       'Asian Restaurant', 'Seafood Restaurant', 'Snack Place',
       'Fish Market', 'Vegetarian / Vegan Restaurant', 'Clothing Store',
       'Hotel', 'Department Store', 'Camera Store', 'Bus Station',
       'Market', 'Shopping Mall', 'Accessories Store', 'Restaurant',
       'Airport Service', 'Maharashtrian Restaurant', 'Food Truck',
       'Nightclub'], dtype=object)

In [44]:
"Shopping Mall" in mumbai_venues['Venue Category'].unique()

True

# 6. Analyize venues in each neighborhood

In [45]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhoods'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Amboli,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [46]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhoods').mean().reset_index()
mumbai_grouped

,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.052632,0.000000,0.00,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632,0.00,0.000000,0.052632,0.00,0.052632,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [47]:
len(mumbai_grouped[mumbai_grouped["Shopping Mall"] > 0])

30

Create new dataframe for Shopping mall data

In [48]:
mumbai_mall = mumbai_grouped[["Neighborhoods","Shopping Mall"]]

In [49]:
mumbai_mall.head()

,Neighborhoods,Shopping Mall
0,Aarey Milk Colony,0.000000
1,Agripada,0.020000
2,Altamount Road,0.000000
3,Amboli,0.011628
4,Amrut Nagar,0.030000


# 7. CLuster neighborhoods

In [50]:
# set number of clusters
kclusters = 3

mumbai_clustering = mumbai_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 2, 2, 0, 0, 2, 1], dtype=int32)

In [51]:
mumbai_merged = mumbai_mall.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

In [52]:
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Aarey Milk Colony,0.000000,0
1,Agripada,0.020000,2
2,Altamount Road,0.000000,0
3,Amboli,0.011628,2
4,Amrut Nagar,0.030000,2


In [53]:
# merge mumbai_grouped with df1 to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(df1.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head() # check the last columns!

(92, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Location,Latitude,Longitude
0,Aarey Milk Colony,0.000000,0,"Goregaon,Western Suburbs",19.148493,72.881756
1,Agripada,0.020000,2,South Mumbai,18.977700,72.827300
2,Altamount Road,0.000000,0,South Mumbai,18.968100,72.809500
3,Amboli,0.011628,2,"Andheri,Western Suburbs",19.129300,72.843400
4,Amrut Nagar,0.030000,2,"Ghatkopar,Eastern Suburbs",19.102077,72.912835


In [54]:
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)

(92, 6)


# 8. Visualize clusters

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 9. View clusters

Cluster: 0 (Red)

In [56]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0].count()

Neighborhood      65
Shopping Mall     65
Cluster Labels    65
Location          65
Latitude          65
Longitude         65
dtype: int64

In [57]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Location,Latitude,Longitude
0,Aarey Milk Colony,0.000000,0,"Goregaon,Western Suburbs",19.148493,72.881756
60,Marine Drive,0.000000,0,South Mumbai,18.944000,72.823000
59,Mankhurd,0.000000,0,Harbour Suburbs,19.050000,72.930000
58,Malabar Hill,0.000000,0,South Mumbai,18.950000,72.795000
57,Mahul,0.000000,0,"Trombay,Harbour Suburbs",19.000000,72.883333
56,Mahim,0.010000,0,South Mumbai,19.035000,72.840000
55,Mahavir Nagar,0.000000,0,"Kandivali West,Western Suburbs",19.211319,72.842737
52,Lokhandwala,0.000000,0,"Andheri,Western Suburbs",19.130815,72.829270
61,Marine Lines,0.000000,0,South Mumbai,18.944700,72.824400
51,Kherwadi,0.000000,0,"Bandra,Western Suburbs",19.055300,72.831400


Cluster: 1 (Blue)

In [58]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1].count()

Neighborhood      7
Shopping Mall     7
Cluster Labels    7
Location          7
Latitude          7
Longitude         7
dtype: int64

In [59]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Location,Latitude,Longitude
9,Bhandup,0.071429,1,Eastern Suburbs,19.1400,72.9300
81,Sunder Nagar,0.040000,1,"Malad,Western Suburbs",19.1750,72.8420
70,Navy Nagar,0.050000,1,"Colaba,South Mumbai",18.9012,72.8101
24,Cuffe Parade,0.038462,1,South Mumbai,18.9100,72.8100
66,Nahur,0.040816,1,"Mulund,Eastern Suburbs",19.1570,72.9410
89,Virar,0.047619,1,Western Suburbs,19.4700,72.8000
48,Kanjurmarg,0.068966,1,Eastern Suburbs,19.1300,72.9400


Cluster: 2 (Light green)

In [60]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2].count()

Neighborhood      20
Shopping Mall     20
Cluster Labels    20
Location          20
Latitude          20
Longitude         20
dtype: int64

In [61]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Location,Latitude,Longitude
83,Thane,0.021277,2,Mumbai,19.200000,72.970000
5,Asalfa,0.030000,2,"Ghatkopar,Eastern Suburbs",19.091000,72.901000
3,Amboli,0.011628,2,"Andheri,Western Suburbs",19.129300,72.843400
82,Thakur village,0.033898,2,"Kandivali East,Western Suburbs",19.210206,72.872980
86,Vidyavihar,0.030000,2,Eastern Suburbs,19.080000,72.896000
1,Agripada,0.020000,2,South Mumbai,18.977700,72.827300
4,Amrut Nagar,0.030000,2,"Ghatkopar,Eastern Suburbs",19.102077,72.912835
74,Pant Nagar,0.016129,2,"Ghatkopar,Eastern Suburbs",19.080000,72.910000
8,Bangur Nagar,0.030000,2,"Goregaon,Western Suburbs",19.167362,72.832252
75,Parel,0.020000,2,South Mumbai,18.990000,72.840000
